In [4]:
from gensim.test.utils import common_texts
import gensim.downloader

In [41]:
import numpy as np
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.lm import Vocabulary
import string
import pickle

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
w2v_mod = gensim.downloader.load('word2vec-google-news-300') # slow

[=================================================-] 99.9% 1661.8/1662.8MB downloaded


In [12]:
# https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92
w2v_mod['fund']
w2v_mod['fund'].shape

(300,)

In [8]:
dir(w2v_mod)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_load_specials',
 '_log_evaluate_word_analogies',
 '_save_specials',
 '_smart_save',
 'accuracy',
 'add',
 'closer_than',
 'cosine_similarities',
 'distance',
 'distances',
 'doesnt_match',
 'evaluate_word_analogies',
 'evaluate_word_pairs',
 'get_keras_embedding',
 'get_vector',
 'index2entity',
 'index2word',
 'init_sims',
 'load',
 'load_word2vec_format',
 'log_accuracy',
 'log_evaluate_word_pairs',
 'most_similar',
 'most_similar_cosmul',
 'most_similar_to_given',
 'n_similarity',
 'rank',
 'save',
 'save_word2vec_format',
 'similar_by_vector',
 's

In [11]:
w2v_mod.most_similar('fund')[:10]

[('funds', 0.8087702393531799),
 ('Fund', 0.6899569034576416),
 ('Funds', 0.6657121181488037),
 ('money', 0.5789710283279419),
 ('NETS_CAC##_Index', 0.5740910768508911),
 ('endowment', 0.5657685995101929),
 ('monies', 0.5621150732040405),
 ('Lawton_Chiles_Endowment', 0.5491019487380981),
 ('funding', 0.5437152981758118),
 ('unearmarked', 0.5397560000419617)]

In [ ]:
# use gensim to create word-to-vec dictionary for austen-emma.


In [14]:
words = nltk.corpus.gutenberg.words('austen-emma.txt')
sentences = nltk.corpus.gutenberg.sents('austen-emma.txt')
my_vocab, index = {}, 1  # start indexing from 1
my_vocab['<pad>'] = 0  # add a padding token
for w in words:
  w = w.lower()
  if w in string.punctuation: continue

  if w not in my_vocab:
    my_vocab[w] = index
    index += 1

my_inverse_vocab = {index: token for token, index in my_vocab.items()}

# sentences, but each word is represented by integers (indices)
ind_sentences = []
for sentence in sentences:
  ind_sentence = []
  for w in sentence:
    w = w.lower()
    if w not in my_vocab: continue

    ind_sentence.append(my_vocab[w])
  ind_sentences.append(ind_sentence)

In [31]:
emb_dim = len(w2v_mod['fund']) # given a word whose index is i, emb_dim[i] is the vector given by w2v_mod
vocab_vec = np.zeros(shape=(len(my_vocab), emb_dim))
vocab_not_in_google_w2v = [] # records the indices of words that w2v_mod didn't include
for i in range(len(my_vocab)):
  w = my_inverse_vocab[i]
  if w not in w2v_mod.vocab:
    vocab_not_in_google_w2v.append(i)
    continue
  vocab_vec[i,] = w2v_mod[w]

In [33]:
len(vocab_not_in_google_w2v)

545

In [42]:
# serialize
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/My Drive/Colab/austen_vocab_vec.pickle', "wb") as outfile:
 # "wb" argument opens the file in binary mode
 pickle.dump(vocab_vec, outfile)

with open('/content/gdrive/My Drive/Colab/austen_vocab_not_in_google_w2v.pickle', "wb") as outfile:
 # "wb" argument opens the file in binary mode
 pickle.dump(vocab_not_in_google_w2v, outfile)


Mounted at /content/gdrive


In [50]:
# to reload, do:
with open('/content/gdrive/My Drive/Colab/austen_vocab_vec.pickle', "rb") as infile:
  vocab_vec_1 = pickle.load(infile)